In [13]:
from code_analysis import CFG, CFGReader
import queue
import time
import numpy as np

In [14]:
##get the nodes for f-open and f-close
reader = CFGReader()
file = "../tp4/part_1/test.php.cfg.json"
graph = reader.read_cfg(file)


In [9]:

def defs(node : int, cfg : CFG) :
    if cfg.get_type(node) == "Variable" :
        #get the parent node
        #look for all BinOP nodes
        child = cfg.get_children(node)[0]
        if cfg.get_type(child) == "BinOP" and cfg.get_image(child) == "=" :
            return set([node])
    return set()

            
def kill_def(node : int, cfg : CFG) :
    defs_nodes = defs(node, cfg)
    if len(defs_nodes) == 0:
        return set()
    to_return = set()
    names_var_defs = set()
    for n in defs_nodes :
        names_var_defs.add(cfg.get_image(n))
    for n in cfg.get_node_ids() :
        if cfg.get_type(n) == "Variable" and cfg.get_image(n) in names_var_defs and n in defs(n,cfg) :
            to_return.add(n)
    return to_return

def refs(node : int, cfg : CFG) :
    if cfg.get_type(node) == "Variable" :
        return set([node]).difference(defs(node, cfg))
    return set()

def possibly_reaching_definition(node_set : set, cfg : CFG) :
    IN : dict[set] = {}
    OUT : dict[set] = {}
    GEN : dict[set] = {}
    KILL : dict[set] = {}
    for node in cfg.get_node_ids() :
        IN[node] = set()
        OUT[node] = set()
        GEN[node] = set()
        KILL[node] = set()
    for node in node_set :
            
        GEN[node] = defs(node, cfg)
        KILL[node] = kill_def(node, cfg)
        
    changes = True
    while changes :
        changes = False
        for node in cfg.get_node_ids() :
            predecessors = cfg.get_parents(node)
            for pred in predecessors :
                IN[node] = IN[node].union(OUT[pred])
            old_out = OUT[node].copy()
            OUT[node] = GEN[node].union(IN[node] - KILL[node])
            if OUT[node] != old_out :
                changes = True
    print("end")
    return IN, OUT, GEN, KILL



def refs(node : int, cfg : CFG) :
    #check if the node is a variable
    
    if cfg.get_type(node) == "Variable" :
        
        return  set([node]).difference(defs(node,cfg))
    return set()

def kill_ref(node : int, cfg : CFG) :
    
    if cfg.get_type(node) == "Variable" :
        defs_node = defs(node, cfg)
        var_name = cfg.get_image(node)
        node_with_same_var_name = set()
        for node_id in cfg.get_node_ids() :
            if cfg.get_type(node_id) == 'Variable' and cfg.get_image(node_id) == var_name :
                node_with_same_var_name.add(node_id)
        return defs_node.intersection(node_with_same_var_name)
            
    return set()
    
def possibly_reachable_reference(node_set : set, cfg : CFG) :
    IN : dict[set] = {}
    OUT : dict[set] = {}
    GEN : dict[set] = {}
    KILL : dict[set] = {}
    for node in cfg.get_node_ids() :
        IN[node] = set()
        OUT[node] = set()
        GEN[node] = set()
        KILL[node] = set()
    for node in node_set :
        GEN[node] = set(refs(node, cfg))
        KILL[node] = set(kill_ref(node, cfg))
        
    changes = True
    while changes :
        changes = False
        for node in cfg.get_node_ids() :
            successors = cfg.get_children(node)
            for suc in successors :
                OUT[node] = OUT[node].union(IN[suc])
            OLD_IN = IN[node].copy()
            IN[node] = GEN[node].union(OUT[node] - KILL[node])
            if IN[node] != OLD_IN :
                changes = True
    print("end")
    return IN, OUT, GEN, KILL
    
            
            

### Verifications Pairs/Définitions Definitions/Pairs

In [10]:

def get_definition_variables_nodes(cfg : CFG) :
    to_return = set()
    for node in cfg.get_node_ids() :
        if cfg.get_type(node) == "BinOP" and cfg.get_image(node) == "=":
            left, _ = cfg.get_op_hands(node)
            if cfg.get_type(left) == "Variable" :
                to_return.add(left)
    return to_return

def get_reference_variables_nodes(cfg : CFG) :
    to_return = set()
    def_var_node = get_definition_variables_nodes(cfg)
    for node in cfg.get_node_ids() :
        if node in def_var_node :
            continue
        if cfg.get_type(node) == "Variable" :
            to_return.add(node)
    return to_return

set_var = get_definition_variables_nodes(graph)

In [21]:
for node_var in set_var :
    print("#######################################")
    print("##### DEFINITION ######")
    print("Node : ", node_var)
    new_set = set()
    new_set.add(node_var)
    IN,OUT,GEN,KILL = possibly_reaching_definition(new_set, graph)
    print("GEN : ", GEN[node_var])
    print("KILL : ", KILL[node_var])
    print("\n")
    #now look for the definition of the variable
    print("##### REFERENCE ######")
    for node in KILL[node_var] :
        print("Node : ", node)
        new_set = set()
        new_set.add(node)
        IN,OUT,GEN,KILL = possibly_reachable_reference(new_set, graph)
        print("GEN : ", GEN[node])
        print("KILL : ", KILL[node])
        print("\n")
        
    

#######################################
##### DEFINITION ######
Node :  10859
end
GEN :  {10859}
KILL :  {10859}


##### REFERENCE ######
Node :  10859
end
GEN :  set()
KILL :  {10859}


#######################################
##### DEFINITION ######
Node :  10862
end
GEN :  {10862}
KILL :  {10862, 10871}


##### REFERENCE ######
Node :  10862
end
GEN :  set()
KILL :  {10862}


Node :  10871
end
GEN :  set()
KILL :  {10871}


#######################################
##### DEFINITION ######
Node :  10871
end
GEN :  {10871}
KILL :  {10862, 10871}


##### REFERENCE ######
Node :  10862
end
GEN :  set()
KILL :  {10862}


Node :  10871
end
GEN :  set()
KILL :  {10871}


